In [1]:
"""
Read VIIRS active fire geolocated fire pixels from XXviirs_access-swath.ipynb
Author: maxwell.cook@colorado.edu
"""

# Import packages
import sys, os
import xarray as xr
import pyproj
import geopandas as gpd
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns

from netCDF4 import Dataset # to open NetCDF
from datetime import datetime
from datetime import timedelta
from matplotlib import pyplot as plt
from affine import Affine
from osgeo import gdal, gdal_array, gdalconst, osr
from rasterio.transform import from_bounds
from scipy.spatial import cKDTree

# Custom functions
sys.path.append(os.path.join(os.getcwd(),'code/'))
from __functions import *

# Projection information
geog = 'EPSG:4326'  # Geographic projection
prj = 'EPSG:5070'  # Projected coordinate system- WGS 84 NAD83 UTM Zone 13N

# File path information
maindir = '/Users/max/Library/CloudStorage/OneDrive-Personal/mcook/'
projdir = os.path.join(maindir, 'aspen-fire/Aim2/')
# Output directories
dataraw = os.path.join(projdir,'data/spatial/raw/VIIRS/')
datamod = os.path.join(projdir,'data/spatial/mod/VIIRS/')

# File path information
print("Ready !")

Ready !


In [22]:
# Load the combined MODIS and VIIRS AFDs (buffered)
fp = os.path.join(datamod,'vnp14img_geo_aspen-fires-srm_2018to2023.csv')
afds = pd.read_csv(fp).reset_index(drop=True)
afds = afds.loc[:, ~afds.columns.str.startswith('Unnamed:')]
len(afds)

79920

In [ ]:
# Create the ground area of pixels

In [23]:
from shapely.geometry import Point

# convert to spatial points using pixel centroid
afds['geometry'] = [Point(xy) for xy in zip(afds.longitude, afds.latitude)]
pix_gdf = gpd.GeoDataFrame(afds, geometry='geometry', crs="EPSG:4326")

# save this file out.
out_fp = os.path.join(datamod,'vnp14img_geo_aspen-fires-srm_pix_latlon.gpkg')
pix_gdf.to_file(out_fp)

# Reproject to a projected CRS for accurate buffering
pix_gdf = pix_gdf.to_crs("EPSG:5070")

# Define the pixel buffer function for the given width and height
def pixel_buffer(point, width, height):
    half_width = width / 2
    half_height = height / 2
    return box(
        point.x - half_width, point.y - half_height,
        point.x + half_width, point.y + half_height
    )

# Apply the buffer function with along_scan and along_track values converted to meters (*1000)
pix_gdf["geometry"] = pix_gdf.apply(
    lambda row: pixel_buffer(row["geometry"], row["along_scan"] * 1000, row["along_track"] * 1000), axis=1
)

pix_gdf = pix_gdf.reset_index(drop=True)
pix_gdf['obs_id'] = pix_gdf.index # unique ID column
# swath_gdf.loc[:, 'acq_datetime'] = swath_gdf.apply(lambda row: convert_datetime(row['acq_date'], row['acq_time']), axis=1)

# save this file out.
out_fp = os.path.join(datamod,'vnp14img_geo_aspen-fires-srm_pix_area.gpkg')
pix_gdf.to_file(out_fp)

pix_gdf.head() # check the results

,longitude,latitude,fire_mask,confidence,daynight,acq_date,acq_time,granule_id,geo_id,frp,iot4,iot5,sample,along_scan,along_track,scan_angle,pix_area,geometry,obs_id
0,-104.60649,38.234726,8,n,Night,6/1/2018,824,VNP14IMG.A2018152.0824.002.2024080110710.nc,VNP03IMG.A2018152.0824,1.722382,304.330841,289.700226,2034,0.565906,0.431471,31.1069,0.244172,"POLYGON ((-744918.548 1722684.619, -744918.548...",0
1,-104.60698,38.232952,8,n,Night,6/1/2018,824,VNP14IMG.A2018152.0824.002.2024080110710.nc,VNP03IMG.A2018152.0824,2.177278,301.303925,288.965240,2035,0.565492,0.431327,31.0802,0.243912,"POLYGON ((-744979.049 1722490.509, -744979.049...",1
2,-104.95000,36.581818,8,n,Night,6/1/2018,824,VNP14IMG.A2018152.0824.002.2024080110710.nc,VNP03IMG.A2018152.0824,1.447277,296.990814,286.114716,2040,0.563433,0.430609,30.9468,0.242619,"POLYGON ((-791995.417 1541023.962, -791995.417...",2
3,-104.94370,36.581154,8,n,Night,6/1/2018,824,VNP14IMG.A2018152.0824.002.2024080110710.nc,VNP03IMG.A2018152.0824,1.447277,298.897369,287.142731,2041,0.563024,0.430467,30.9202,0.242363,"POLYGON ((-791446.567 1540897.474, -791446.567...",3
4,-104.99421,36.582436,8,n,Night,6/1/2018,824,VNP14IMG.A2018152.0824.002.2024080110710.nc,VNP03IMG.A2018152.0824,4.948321,323.709015,290.587463,2033,0.566321,0.431616,31.1336,0.244433,"POLYGON ((-795889.355 1541461.861, -795889.355...",4


In [24]:
len(pix_gdf)

79920

In [ ]:
# Load and tidy the fire perimeter data
fp = os.path.join(projdir,'data/spatial/mod/NIFC/nifc-ics_2018_to_2023-aspen.gpkg')
fires = gpd.read_file(fp)

# subset to Southern Rockies
fires = fires[fires['na_l3name'] == 'Southern Rockies']

# tidy the fire id and name columns
fires.rename(columns={'NIFC_ID': 'Fire_ID', 'NIFC_NAME': 'Fire_Name'}, inplace=True)
# tify the date columns
fires['DISCOVERY_DATE'] = pd.to_datetime(fires['DISCOVERY_DATE']).dt.date
fires['WF_CESSATION_DATE'] = pd.to_datetime(fires['WF_CESSATION_DATE']).dt.date

fires['NIFC_ACRES'] = fires['NIFC_ACRES'].astype(float)
fires = fires[fires['NIFC_ACRES'] > 34.749]
print(len(fires))

In [ ]:
# Add a small buffer to wildfire perimeters
buffer_dist = 1000  # meters
fires_ = fires.copy() # make a copy of the original data
fires_['geometry'] = fires_['geometry'].buffer(buffer_dist)
print(f"Buffered fire perimeters by {buffer_dist} meters.")

In [ ]:
# Perform spatial join with AFDs
afds_ = gpd.sjoin(pix_gdf, fires_, how='inner', predicate='intersects')
afds_.drop(columns=['index_right'], inplace=True)
afds_.head()

In [ ]:
# # Subset to the Southern Rockies (for now) and VIIRS
# afds_srm = afds[afds['na_l3name'] == 'Southern Rockies'] # subset to SRM
# afds_srm = afds_srm[afds_srm['INSTRUMENT'] == 'VIIRS'] # VIIRS detections
# print(f"There are [{len(afds_srm['NIFC_ID'].unique())}/{len(afds['NIFC_ID'].unique())}] fire events in the Southern Rockies (2018-2023).")

In [ ]:
# print(afds_srm['FRP'].isna().sum())

In [ ]:
# Handle duplicate observations

In [ ]:
# Case 1. Spatial overlap > 90% and the same acquisition time

In [ ]:
# Find duplicates in space and time
drop_obs = set() # to store the observations we want to drop
for idx, obs in afds_srm.iterrows():
    afdID_ = obs['afdID']
    date_matches = afds_srm[
        (afds_srm['ACQ_DATETIME'] == obs['ACQ_DATETIME']) &
        (afds_srm.afdID != afdID_)
    ]
    if len(date_matches) > 0:
        for match_idx, match_obs in date_matches.iterrows():
            afdID_match = match_obs['afdID']
            # Calculate overlap
            overlap_area = obs.geometry.intersection(match_obs.geometry).area
            ratio = overlap_area / obs.geometry.area
            # Check for >50% overlap
            if ratio > 0.50:
                # Keep the one with higher FRP, mark lower FRP for removal
                if match_obs['FRP'] < obs['FRP']:
                    drop_obs.add(afdID_match)

afds_srm_c1 = afds_srm[~afds_srm['afdID'].isin(drop_obs)]
print(f"Identified a total of [{len(drop_obs)}/{len(afds_srm)}] duplicate observations.")

out_fp = os.path.join(projdir,'data/spatial/mod/VIIRS/afd_aspen-fires_Case1.gpkg')
afds_srm_c1.to_file(out_fp)
print(f"Saved to: {out_fp}")

In [ ]:
print(afds_srm_c1['FRP'].isna().sum())

In [ ]:
# Case 2: Greater than 50% overlap, same day, different time.

In [ ]:
time_threshold = timedelta(hours=3) # search for time within 3 hours

afds_srm_c1 = afds_srm_c1.reset_index()

drop_obs = set()
for acq_date, group in afds_srm_c1.groupby('ACQ_DATE'):
    for dn in ['D','N']:
        dn_group = group[group['DAYNIGHT'] == dn].sort_values('ACQ_DATETIME')
        for idx, obs in dn_group.iterrows():
            time_matches = dn_group[
                (dn_group['ACQ_DATETIME'] > obs['ACQ_DATETIME']) &
                (dn_group['ACQ_DATETIME'] <= obs['ACQ_DATETIME'] + time_threshold) &
                (dn_group.index != idx)  # Exclude itself
            ]
            for match_idx, match_obs in time_matches.iterrows():
                # Calculate spatial overlap ratio
                overlap_area = obs.geometry.intersection(match_obs.geometry).area
                overlap_ratio = overlap_area / obs.geometry.area
                
                # Check for >50% spatial overlap
                if overlap_ratio > 0.50:
                    # Retain the highest FRP observation
                    if match_obs['FRP'] < obs['FRP']:
                        drop_obs.add(match_idx)

afds_srm_c2 = afds_srm_c1.drop(index=list(drop_obs)).reset_index(drop=True)
print(f"Identified a total of [{len(drop_obs)}/{len(afds_srm_c1)}] overlapping observations within 2 hours, separated by day/night.")

out_fp = os.path.join(projdir, 'data/spatial/mod/VIIRS/afd_aspen-fires_Case2.gpkg')
afds_srm_c2.to_file(out_fp)
print(f"Saved to: {out_fp}")

In [ ]:
print(afds_srm_c2['FRP'].isna().sum())

In [ ]:
# Case 3: Classify first day of burn and secondary

In [ ]:
afds_srm_c2['detection'] = 'primary' # set the first day of burn / maximum FRP
afds_srm_c3 = afds_srm_c2.sort_values(['ACQ_DATETIME']).reset_index(drop=True)
afds_srm_c3 = afds_srm_c3.reset_index(drop=True)
afds_srm_c3.drop(columns=['index'], inplace=True)

overlap_gdf = gpd.sjoin(
    afds_srm_c3, 
    afds_srm_c3, 
    how="inner", 
    predicate="intersects", 
    lsuffix="left", 
    rsuffix="right")

# Iterate over each observation in swath_gdf_c3
for idx, obs in afds_srm_c3.iterrows():
    # Filter to previous overlapping observations only
    overlapping_obs = overlap_gdf[
        (overlap_gdf['afdID_left'] == obs['afdID']) &  # Current observation
        (overlap_gdf['afdID_right'] < obs['afdID']) &  # Previous observations only
        (overlap_gdf['DAYNIGHT_right'] == obs['DAYNIGHT'])  # Same day/night cycle
    ].copy()

    # Calculate overlap ratio for each overlapping observation
    overlap_ratios = [
        obs.geometry.intersection(afds_srm_c3.loc[afds_srm_c3['afdID'] == row['afdID_right']].geometry.iloc[0]).area
        / obs.geometry.area
        for _, row in overlapping_obs.iterrows()
    ]

    # Filter to those with significant overlap (e.g., >50%)
    overlapping_obs['overlap_ratio'] = overlap_ratios
    overlapping_obs = overlapping_obs[overlapping_obs['overlap_ratio'] > 0.5]

    # If there are overlapping observations, proceed to classify
    if not overlapping_obs.empty:
        max_frp = overlapping_obs['FRP_right'].max()  # Max FRP from overlapping observations

        # Classify based on FRP comparison
        if obs['FRP'] > max_frp:
            afds_srm_c3.at[idx, 'detection'] = 'flare-up'
        else:
            afds_srm_c3.at[idx, 'detection'] = 'secondary'

out_fp = os.path.join(projdir, 'data/spatial/mod/VIIRS/afd_aspen-fires_Case3.gpkg')
afds_srm_c3.to_file(out_fp)
print(f"Saved to: {out_fp}")

In [ ]:
print(afds_srm_c3['FRP'].isna().sum())

In [ ]:
afds_srm_c3['detection'].unique()

In [ ]:
# Plot FRP for each detection label
plt.figure(figsize=(5.5, 3))

sns.boxplot(data=afds_srm_c3, x='detection', y='log_FRP', palette="inferno")
plt.title("Distribution of log(FRP) by Detection Type")
plt.xlabel("Detection Type")
plt.ylabel("log(FRP)")

plt.savefig(os.path.join(projdir,'figures/FigureX_Primary_Secondary-FRP.png'), dpi=150, bbox_inches='tight')

plt.show()

In [ ]:
# Plot the distribution of "front" and "re-burn"
burn_class_counts = afds_srm_c3['detection'].value_counts()

# Plot a bar chart
plt.figure(figsize=(5, 3))
sns.barplot(x=burn_class_counts.index, y=burn_class_counts.values, palette="inferno")
plt.title("Primary, Secondary, and Flare-up Obs.")
plt.xlabel("Burn Class")
plt.ylabel("Count")

plt.savefig(os.path.join(projdir,'figures/FigureX_Primary_Secondary-Counts.png'), dpi=150, bbox_inches='tight')

plt.show()

In [ ]:
len(afds_srm_c3)

In [ ]:
# Join to CBI, TreeMap, etc

In [ ]:
# Load the summary table (from GEE reductions)
fp = os.path.join(projdir,'data/tabular/mod/AFD/combined-afd_aspen-fires_TreeMap.csv')
treemap = pd.read_csv(fp)
treemap.drop(columns=["Unnamed: 0"], inplace=True)
print(len(treemap['afdID'].unique()))
treemap.head()

In [ ]:
# Identify "forested" VIIRS observations
# Calculate the total forest cover percent
pct_cover_sum = treemap.groupby('afdID')['pct_cover'].sum()
# Filter afdIDs with a sum of pct_cover greater than 50%
afdIDs_gt50 = pct_cover_sum[pct_cover_sum > 50].index

# Filter the original dataframe to keep only rows with these afdIDs
treemap_ = treemap[treemap['afdID'].isin(afdIDs_gt50)]
print(f"Number of afdIDs with pct_cover > 50%: {len(afdIDs_gt50)}")

In [ ]:
# Identify the dominant forest type
dtree = treemap_.loc[treemap_.groupby('afdID')['pct_cover'].idxmax()]
print(len(dtree))
dtree.head()

In [ ]:
dtree_frp = dtree.merge(afds_srm_c3, on='afdID', how='inner')
print(dtree_frp.columns)
len(dtree_frp)

In [ ]:
print(dtree_frp['FRP'].isna().sum())

In [ ]:
### Plot FRP by EVT group name
df = dtree_frp[dtree_frp['DAYNIGHT'] == 'D'] # primary obs.
df = dtree_frp

# Find the median and sort the evt classes
med = df.groupby('species_name')['FRP'].median().sort_values(ascending=False)
sorted = med.index.tolist()

plt.figure(figsize=(8, 8)) 

sns.boxplot(
    data=df,
    x='FRP', 
    y='species_name', 
    hue='detection',
    order=sorted,  
    palette='coolwarm', 
    dodge=True
)

plt.xscale('log')

plt.title('Daytime FRP by Detection Class and Species')
plt.xlabel('Fire Radiative Power (FRP)')
plt.ylabel('Dominant Tree Species')

plt.tight_layout()

plt.savefig(os.path.join(projdir,'figures/FigureX_DTree-FRP-Detections.png'), dpi=300, bbox_inches='tight')

plt.show()

In [ ]:
aspen_df = dtree_frp[dtree_frp['species_name'] == 'Aspen']
aspen_df = aspen_df[aspen_df['DAYNIGHT'] == 'D']

# Create the scatter plot
plt.figure(figsize=(6, 4))
sns.scatterplot(data=aspen_df, x='FRP', y='BALIVE', hue='detection', style='detection', s=50, palette="viridis")
plt.xlabel('Aspen Live Basal Area')
plt.ylabel('FRP')
plt.legend(title='Detection Type')

plt.savefig(os.path.join(projdir,'figures/FigureX_DTree-Aspen-FRP_scatter.png'), dpi=300, bbox_inches='tight')

plt.show()

In [ ]:
# Join in the CBI
cbi = os.path.join(projdir,'data/tabular/mod/AFD/combined-afd_aspen-fires_cbi.csv')
cbi = pd.read_csv(cbi)
cbi.columns

In [ ]:
cbi = cbi[['afdID','CBI_mean', 'CBI_bc_mean', 'rbr_mean']]
cbi.head()

In [ ]:
dtree_frp_cbi = dtree_frp.merge(cbi, on='afdID', how='inner')
print(len(dtree_frp_cbi))
dtree_frp_cbi.head()

In [ ]:
dtree_frp_cbi = dtree_frp_cbi.drop_duplicates(subset='afdID', keep='first')
len(dtree_frp_cbi)

In [ ]:
print(dtree_frp_cbi['afdID'].isna().sum())
print(dtree_frp_cbi['CBI_mean'].isna().sum())
print(dtree_frp_cbi['species_name'].isna().sum())
print(dtree_frp_cbi['FRP'].isna().sum())

In [ ]:
# Calculate the Pearson correlation by forest type

In [ ]:
df = dtree_frp_cbi[dtree_frp_cbi['detection'] == 'primary'] #primary obs.
df = dtree_frp_cbi[dtree_frp_cbi['DAYNIGHT'] == 'D'] #daytime

In [ ]:
from scipy.stats import pearsonr

correlations = {}
for tree_type, group in df.groupby('species_name'):
    corr, p_value = pearsonr(group['FRP'], group['CBI_bc_mean'])
    correlations[tree_type] = (corr, p_value)
    print(f"Tree Type: {tree_type} - Pearson correlation: {corr:.2f}, p-value: {p_value:.4f}")

In [ ]:
correlation_df = pd.DataFrame(correlations, index=['correlation', 'p_value']).T
print("\nCorrelation by Tree Type:\n")
correlation_df = correlation_df.sort_values('correlation')
correlation_df.to_csv(os.path.join(projdir, 'data/tabular/mod/afds_aspen-fires_frp-cbi_pearson.csv'))
correlation_df.head(15)

In [ ]:
g = sns.lmplot(data=df, x='FRP', y='CBI_bc_mean', hue='species_name', col='species_name', 
           col_wrap=3, height=2, aspect=1.5, scatter_kws={'alpha':0.4}, line_kws={'color':'black'}, sharey=True
          )

# Adjust the titles to remove the "species_name = " prefix
for ax in g.axes.flat:
    ax.set_ylim(0, 4)  # Set the y-axis limit from 0 to 4
    ax.set_title(ax.get_title().replace('species_name = ', ''))

plt.savefig(os.path.join(projdir,'figures/FigureX_DTree-FRP-CBI_bc.png'), dpi=300, bbox_inches='tight')

plt.show()